# Logistic Regression Consulting Project

## Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

In [185]:
import findspark

In [186]:
findspark.init('/home/pushya/spark-2.1.0-bin-hadoop2.7')

In [187]:
import pyspark

In [188]:
from pyspark.sql import SparkSession

In [189]:
spark = SparkSession.builder.appName('mylogreg').getOrCreate()

In [190]:
from pyspark.ml.classification import LogisticRegression

In [191]:
# Load training data
df = spark.read.csv("customer_churn.csv",inferSchema=True,header=True)

In [131]:
df.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [192]:
colums = df.select(
    
    
 'Age',
 'Total_Purchase',
 'Years',
 'Num_Sites',
 'Company',
 'Churn'                  )

In [195]:
my_final_data = colums.na.drop()

In [196]:
from pyspark.ml.feature import (VectorAssembler,VectorIndexer,
                       OneHotEncoder,StringIndexer)

In [198]:
assembler = VectorAssembler(inputCols=[
    'Age',
 'Total_Purchase',
 'Years',
 'Num_Sites',
   ],outputCol='features')

In [201]:
final_data=assembler.transform(my_final_data)

In [202]:
from pyspark.ml.classification import LogisticRegression

In [203]:
train_data, test_data = final_data.randomSplit([0.7,.3])

In [204]:
log_reg_churn = LogisticRegression(featuresCol='features',labelCol='Churn')

In [148]:
pipeline = Pipeline(stages=[gender_indexer,
                           gender_encoder,
                           assembler,log_reg_churn])

In [205]:
fit_model = log_reg_churn.fit(train_data)

In [206]:
results = fit_model.transform(test_data)

In [207]:
results.select('Churn','prediction').show()

+-----+----------+
|Churn|prediction|
+-----+----------+
|    1|       0.0|
|    0|       0.0|
|    0|       0.0|
|    1|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    1|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
+-----+----------+
only showing top 20 rows



In [208]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [209]:
my_eval = BinaryClassificationEvaluator(rawPredictionCol='rawPrediction',
    labelCol='Churn')

In [210]:
my_eval.evaluate(results)

0.8887188902488763

In [211]:
# Load training data
new_customers = spark.read.csv("new_customers.csv",inferSchema=True,header=True)

In [212]:
new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [213]:
final_lr_model = log_reg_churn.fit(final_data)

In [214]:
test_new_customers = assembler.transform(new_customers)

In [215]:
test_new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- features: vector (nullable = true)



In [216]:
final_results = final_lr_model.transform(test_new_customers)

In [217]:
final_results.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = true)



In [218]:
final_results.show()

+--------------+----+--------------+---------------+-----+---------+--------------------+--------------------+----------------+--------------------+--------------------+--------------------+----------+
|         Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|        Onboard_date|            Location|         Company|            features|       rawPrediction|         probability|prediction|
+--------------+----+--------------+---------------+-----+---------+--------------------+--------------------+----------------+--------------------+--------------------+--------------------+----------+
| Andrew Mccall|37.0|       9935.53|              1| 7.71|      8.0|2011-08-29 18:37:...|38612 Johnny Stra...|        King Ltd|[37.0,9935.53,7.7...|[2.41832366013597...|[0.91821394450559...|       0.0|
|Michele Wright|23.0|       7526.94|              1| 9.28|     15.0|2013-07-22 18:19:...|21083 Nicole Junc...|   Cannon-Benson|[23.0,7526.94,9.2...|[-5.9906678174824...|[0.00249574827398...|  

In [219]:
final_results.select('Company','prediction').show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+

